In [14]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

In [15]:
# physical_devices=tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [16]:
(ds_train,ds_test), ds_info=tfds.load(
"mnist",
split=["train","test"],
shuffle_files=True,
as_supervised=True,
with_info=True,
)

In [17]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [18]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [19]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [20]:
# Setup for test Dataset
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

In [21]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

In [22]:
num_epochs = 5
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [23]:
# training loop
for epoch in range(num_epochs):
    print(f"\n start of training epoch {epoch}")
    for batch_idx,(x_batch,y_batch) in enumerate(ds_train):
        with tf.GradientTape() as tape:
            y_pred=model(x_batch,training=True)
            loss=loss_fn(y_batch,y_pred)
        gradients=tape.gradient(loss,model.trainable_weights)
        optimizer.apply_gradients(zip(gradients,model.trainable_weights))
        acc_metric.update_state(y_batch,y_pred)
train_acc=acc_metric.result()
print(f"Accuracy over epoch {train_acc}")
acc_metric.reset_states()


 start of training epoch 0

 start of training epoch 1

 start of training epoch 2

 start of training epoch 3

 start of training epoch 4
Accuracy over epoch 0.9726633429527283


In [24]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
    y_pred = model(x_batch, training=False)
    acc_metric.update_state(y_batch, y_pred)

In [25]:
test_acc = acc_metric.result()
print(f"Accuracy over Test Set: {test_acc}")
acc_metric.reset_states()

Accuracy over Test Set: 0.992900013923645
